# Titanic Survival Analysis

<img src="https://upload.wikimedia.org/wikipedia/commons/d/db/Titanic-Cobh-Harbour-1912.JPG"/ width=500/>

We want to examine survival rates on the RMS Titanic, and more importantly, who was it that survived? Given that we want to know the **features** that contributed to one's survival on that fateful shipwreck, **decision trees** seem quite suited to this purpose.

Let's take a look at some of the variables in the dataset.

------

**VARIABLE DESCRIPTIONS:**
* survival   -     Survival
                (0 = No; 1 = Yes)
* pclass     -     Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
* name       -     Name
* sex        -     Sex
* age        -     Age
* sibsp      -     Number of Siblings/Spouses Aboard
* parch      -     Number of Parents/Children Aboard
* ticket     -     Ticket Number
* fare       -     Passenger Fare
* cabin      -     Cabin
* embarked   -     Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

**SPECIAL NOTES:**
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

* Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
* Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
* Parent:   Mother or Father of Passenger Aboard Titanic
* Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt

%matplotlib inline

# Imports to allow us to write to io and view images in ipython
from IPython.display import Image
from sklearn.externals.six import StringIO
import pydot

In [ ]:
path_to_repo = "/Users/brianchung/Desktop/ga-ds/"
data = pd.read_csv(path_to_repo + "/12_trees/titanic.csv")

In [ ]:
data.head()

--------
If you watch "disaster" based movies, one thing they always say is "Take the women and children first!" Let's see how true that is. 

Does being a woman and/or child help in having survived the titanic crash?

In [ ]:
X = pd.DataFrame({'Age': data.Age.replace([np.NaN],data.Age.median(method='drop')),
                  'Sex': data.Sex.apply(lambda x: 1 if x=="male" else 0)})
y = data.Survived

In [ ]:
X = X[ ~y.isnull() ]
y = y[ ~y.isnull() ]

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(X,y)
print model.score(X,y), model.classes_
print model.feature_importances_

# Write a decision tree to a stringIO buffer using tree.export_graphviz
dot_data = StringIO()
tree.export_graphviz(model,out_file=dot_data,feature_names=["Age","Sex"])

# From the stringio, create a graph and output to screen
graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())



In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

np.mean(cross_val_score( tree.DecisionTreeClassifier(),X,y,scoring="roc_auc",cv=KFold(len(X),n_folds=10,shuffle=True)))

Seems it was not a good time to have that Y chromosome.

------
# Exercise

Try improving upon the decision tree score for roc_auc. Recall that the roc_auc (Receiver Operating Characteristic Area under the curve) can be used as a metric in binary classification problems.

One feature that might help improve the score is the "Pclass" metric. Perhaps the rich were shuttled off the boat first? Report your scores using Decision trees, and what factors were most important in determining survival.